In [357]:
from data import Data
from methods import MABMethod
import numpy as np
import math
import matplotlib.pyplot as plt

In [358]:
## Define radius
def raddist(x,y):
    return math.sqrt((x-.5)**2+(y-.5)**2)
def isAnomaly(x,y):
    return (raddist(x,y)>.5)
def label(instance):
    if (isAnomaly(instance[0], instance[1])):
        return 1
    else:
        return -1


## Create toy data set
domain_1 = np.random.default_rng(seed=125).random((35,2))
domain_2 = np.random.default_rng(seed=45).random((35,2))
anomalies_1 = []
anomalies_2 = []
normals_1 = []
normals_2 = []

for instance in domain_1:
    if isAnomaly(instance[0], instance[1]):
        anomalies_1.append(instance)
    else:
        normals_1.append(instance)

for instance in domain_2:
    if isAnomaly(instance[0], instance[1]):
        anomalies_2.append(instance)
    else:
        normals_2.append(instance)

anomalies_1 = np.asarray(anomalies_1)
anomalies_2 = np.asarray(anomalies_2)
normals_1 = np.asarray(normals_1)
normals_2 = np.asarray(normals_2)


fig, ax = plt.subplots( nrows=1, ncols=1 )  # create figure & 1 axis
ax.scatter(normals_1[:,0], normals_1[:,1], c= 'b')
ax.scatter(normals_2[:,0], normals_2[:,1], c= 'g')
if (len(anomalies_1)>0):
    ax.scatter(anomalies_1[:,0], anomalies_1[:,1], c= 'b', marker='+')
if (len(anomalies_2)>0):
    ax.scatter(anomalies_2[:,0], anomalies_2[:,1], c= 'g', marker='+')
fig.savefig('img/original.png',bbox_inches='tight')
plt.close(fig)



In [359]:
def getPrediction(alba: MABMethod, data: Data):
    pos = {}
    neg = {}
    predictions = alba.predict(data)
    for key in predictions.keys():
        posdummy = []
        negdummy = []
        for i in range(len(predictions[key])):
            if predictions[key][i]==-1:
                negdummy.append(data.get_domain(key)[i])
            elif predictions[key][i]==1:
                posdummy.append(data.get_domain(key)[i])
        pos[key] = np.asarray(posdummy)
        neg[key] = np.asarray(negdummy)
    return neg, pos
def currentPrediction(alba, data, labeledPos = None, labeledNeg = None, t=0):
    neg, pos = getPrediction(alba, data)
    neg_1, neg_2 = neg.values()
    pos_1, pos_2 = pos.values()
    fig, ax = plt.subplots( nrows=1, ncols=1 )  # create figure & 1 axis
    if (len(neg_1)>0):
        ax.scatter(neg_1[:,0], neg_1[:,1], c= 'b')
    if (len(neg_2)>0):
        ax.scatter(neg_2[:,0], neg_2[:,1], c= 'g')
    if (len(pos_1)>0):
        ax.scatter(pos_1[:,0], pos_1[:,1], c= 'b', marker='+')
    if (len(pos_2)>0):
        ax.scatter(pos_2[:,0], pos_2[:,1], c= 'g', marker='+')
    if (len(labeledPos)>0):
        ax.scatter(labeledPos[:,0], labeledPos[:,1], c = 'r', marker = '+')
    if (len(labeledNeg)>0):
        ax.scatter(labeledNeg[:,0], labeledNeg[:,1], c = 'r', marker = 'o')
    fig.savefig('img/iteration'+str(t).zfill(3)+'.png',bbox_inches='tight')
    plt.close(fig)

In [360]:
## Create 2 domains
training_data = Data(2)
training_data.set_domains_and_labels({0:domain_1, 1:domain_2})
query_budget = 70
alba = MABMethod(mab="rotting-swa", query_budget=query_budget, verbose=True )


In [361]:
t = 0  
labeledPos = []
labeledNeg = []

import warnings
warnings.filterwarnings('ignore')

while(t<query_budget):
    queries = alba.fit_query(training_data, True)

    key,idx = queries[0]
    instance = training_data.get_domain(key)[idx]
    training_data.set_new_label(key, idx, label(instance))
    training_data.set_last_labeled(key,idx)
    if (label(instance) == 1):
        labeledPos.append(instance)
    else:
        labeledNeg.append(instance)


    currentPrediction(alba, training_data, np.asarray(labeledPos), np.asarray(labeledNeg), t)

    t += 1

In [362]:
import cv2
import os

image_folder = 'img'
video_name = 'video.avi'

images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
print(images)
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

video = cv2.VideoWriter(video_name, 0, 1, (width,height))

for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

video.release()

['iteration000.png', 'iteration001.png', 'iteration002.png', 'iteration003.png', 'iteration004.png', 'iteration005.png', 'iteration006.png', 'iteration007.png', 'iteration008.png', 'iteration009.png', 'iteration010.png', 'iteration011.png', 'iteration012.png', 'iteration013.png', 'iteration014.png', 'iteration015.png', 'iteration016.png', 'iteration017.png', 'iteration018.png', 'iteration019.png', 'iteration020.png', 'iteration021.png', 'iteration022.png', 'iteration023.png', 'iteration024.png', 'iteration025.png', 'iteration026.png', 'iteration027.png', 'iteration028.png', 'iteration029.png', 'iteration030.png', 'iteration031.png', 'iteration032.png', 'iteration033.png', 'iteration034.png', 'iteration035.png', 'iteration036.png', 'iteration037.png', 'iteration038.png', 'iteration039.png', 'iteration040.png', 'iteration041.png', 'iteration042.png', 'iteration043.png', 'iteration044.png', 'iteration045.png', 'iteration046.png', 'iteration047.png', 'iteration048.png', 'iteration049.png',